In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os, re, csv, math, codecs, pickle
from statsmodels.stats.contingency_tables import mcnemar

In [2]:
with open('fastTextCNN_predicted_labels.pkl', 'rb') as file:
    CNN_predicted_labels = pickle.load(file)
    print(CNN_predicted_labels)

[[1]
 [0]
 [0]
 ...
 [1]
 [1]
 [1]]


In [3]:
with open('TFIDFFFNN_predicted_labels.pkl', 'rb') as file:
    FFNN_predicted_labels = pickle.load(file)
    print(FFNN_predicted_labels)

[1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 

In [4]:
with open('true_labels.pkl', 'rb') as file:
    true_labels = pickle.load(file)
    print(true_labels)

[1 0 1 ... 1 1 1]


In [5]:
CNN_predicted_labels = CNN_predicted_labels.ravel()
CNN_predicted_labels

array([1, 0, 0, ..., 1, 1, 1])

In [6]:
df_models = pd.crosstab([CNN_predicted_labels,FFNN_predicted_labels], true_labels, rownames=['CNN', 'FFNN'], colnames=['True Label'], margins=True)
df_models

True Label    0    1   All
CNN FFNN                  
0   0       406   72   478
    1        88  148   236
1   0         6   19    25
    1        22  263   285
All         522  502  1024

In [7]:
A = 0 #both correct
B = 0 #cnn incorrect, ffnn correct
C = 0 #ffnn incorrect, cnn correct
D = 0 #both wrong

for i in range(len(true_labels)):
    # n11 : number of items classified correctly by both FFNN and CNN
    if (FFNN_predicted_labels[i] == true_labels [i] and CNN_predicted_labels[i] == true_labels [i]):
        A = A + 1
        
    # n00: number of items misclassified by both FFNN and CNN
    elif (FFNN_predicted_labels[i] != true_labels [i] and CNN_predicted_labels[i] != true_labels [i]):
        D = D + 1
        
    # n10: number of items misclassified by CNN but not by FFNN
    elif (FFNN_predicted_labels[i] == true_labels [i] and CNN_predicted_labels[i] != true_labels [i]):
        B = B + 1
    
    # n01: number of items misclassified by FFNN but not by CNN
    elif (FFNN_predicted_labels[i] != true_labels [i] and CNN_predicted_labels[i] == true_labels [i]):
        C = C + 1
        
print("Number of items classified correctly by both FFNN and CNN: ", A) 
print("Number of items misclassified by CNN but not by FFNN: ", B)
print("Number of items misclassified by FFNN but not by CNN: ", C)
print("Number of items misclassified by both FFNN and CNN: ", D)
print("Total number of items: ", A+B+C+D)

Number of items classified correctly by both FFNN and CNN:  669
Number of items misclassified by CNN but not by FFNN:  154
Number of items misclassified by FFNN but not by CNN:  107
Number of items misclassified by both FFNN and CNN:  94
Total number of items:  1024


In [8]:
# define contingency table
table = [[D, C],
		 [B, A]]

In [9]:
# Hypothesis Testing
alpha = 0.05

# McNemar's Test with the continuity correction
test = mcnemar(table, exact=True, correction=True)

if test.pvalue < alpha:
  print("Reject Null hypotesis")
else:
  print("Fail to reject Null hypotesis")
print("P-value: ", test.pvalue)

Reject Null hypotesis
P-value:  0.004319137524752679


In [10]:
# n = number of dependent tests
n = 3
# rank of the p-value from smallest to greatest of the tests
rank = 1

# Holm-Bonferroni Formula
HB = alpha / (n - rank + 1)

if test.pvalue < HB:
  print("Reject Null hypotesis")
else:
  print("Fail to reject Null hypotesis")
print("Holm-Bonferroni correction: ", HB)

Reject Null hypotesis
Holm-Bonferroni correction:  0.016666666666666666
